In [11]:
import numpy as np
import pandas as pd


In [2]:
node = pd.read_table("./node.txt",delimiter=",")
node["osmid"].astype("int64")
node = node[["osmid","y","x"]]
node

,osmid,y,x
0,2.558506e+07,39.901987,116.385578
1,2.558508e+07,39.899003,116.386792
2,2.558509e+07,39.898860,116.381862
3,2.558512e+07,39.900544,116.377850
4,2.558512e+07,39.902940,116.377763
...,...,...,...
37481,1.007778e+10,39.996019,116.222735
37482,1.009133e+10,39.896058,116.599433
37483,1.009133e+10,39.773362,116.173259
37484,1.009133e+10,39.775494,116.169075


In [3]:
node_info = {}
num = 0
for i in range(node.shape[0]):
    # print(round(node["osmid"][i]))
    node_info[round(node["osmid"][i])] = [num,node["y"][i],node["x"][i]]
    num = num+1

In [18]:
edge = pd.read_table("./edge.txt",delimiter=",")
edge = edge[["FID","u","v"]]
# node["osmid"].astype("int64")
edge

e:\Anaconda\envs\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,FID,u,v
0,0,2.558506e+07,5.336507e+08
1,1,2.558506e+07,8.835881e+09
2,2,2.558506e+07,3.427913e+08
3,3,2.558508e+07,1.414035e+09
4,4,2.558508e+07,9.378036e+09
...,...,...,...
86311,86311,1.009133e+10,4.076207e+09
86312,86312,1.009133e+10,5.345749e+08
86313,86313,1.009133e+10,9.783252e+09
86314,86314,1.009133e+10,1.009133e+10


In [19]:
np.unique(edge["u"].to_numpy()).shape
# 

(37404,)

In [20]:
import math
"""所谓方位角是与正北方向、顺时针之间的夹角"""
def calc_azimuth(lat1, lon1, lat2, lon2):
    lat1_rad = lat1 * math.pi / 180
    lon1_rad = lon1 * math.pi / 180
    lat2_rad = lat2 * math.pi / 180
    lon2_rad = lon2 * math.pi / 180

    y = math.sin(lon2_rad - lon1_rad) * math.cos(lat2_rad)
    x = math.cos(lat1_rad) * math.sin(lat2_rad) - \
        math.sin(lat1_rad) * math.cos(lat2_rad) * math.cos(lon2_rad - lon1_rad)

    brng = math.atan2(y, x) * 180 / math.pi

    return float((brng + 360.0) % 360.0)

In [21]:
edge_dir_emb = []

K = 8
W = 360/K

for index,item in edge.iterrows():
    """
    FID            0.0
    u       25585055.0
    v      533650673.0
    Name: 0, dtype: float64
    """
    # print(round(item['v']))
    # print(node_info[round(item['v'])])
    """
    [39.9019869, 116.3855782]
    """
    temp = []
    temp.append(int(item["FID"]))
    # temp.append(round(item["u"]))
    temp.append(node_info[round(item["u"])][0])  # start node 
    # temp.append(round(item["v"]))
    temp.append(node_info[round(item["v"])][0])   # end node 
    dir_angle = calc_azimuth(node_info[round(item['u'])][1],node_info[round(item['u'])][2],node_info[round(item['v'])][1],node_info[round(item['v'])][2])
    # print(int(dir_angle))
    temp.append(int(dir_angle))

    dir_interval = int(dir_angle/W)
    # print(dir_interval)
    temp.append(dir_interval)

    edge_dir_emb.append(temp)

    # break


In [12]:
# edge_dir_emb = np.array(edge_dir_emb)
# edge_dir_emb
# edge_dir_emb[:,1]
# len(np.unique(edge_dir_emb[:,1]))
# np.save("./1114/edge_dir_emb.npy",edge_dir_emb)

# """
# edgeid，u，v,v_id,方向角,象限  ,
# array([[    0,     0,  4046,   106,     2],
#        [    1,     0, 29680,   359,     7],
#        [    2,     0,  2488,   191,     4],
#        ...,
#        [86313, 37484, 36319,    36,     0],
#        [86314, 37484, 37485,   127,     2],
#        [86315, 37485, 37484,   307,     6]])
# """
edge_dir_emb = pd.read_csv("./1114/edge_dir_emb-t.csv")
a = edge_dir_emb
# a = pd.DataFrame(edge_dir_emb)
# print(a["1"])
# a.to_csv("./1114/edge_dir_emb.csv",index=None)
total = 0
for i in range(37486):
    temp = a[a["1"]==i].to_numpy()[:,4]
    if(len(temp)==0):
        continue
    # print(temp)
    if(len(temp)!=len(np.unique(temp))):
        total+=1
        # print(len(temp)-len(np.unique(temp)))
        # print(i)
    # break

In [17]:
np.save("./1114/edge_dir_emb.npy",a.to_numpy())

In [35]:
## 得到 edgeid_node_dir_map
edge_dir_emb = edge_dir_emb.to_numpy()
# print(edge_dir_emb)
edgeid_node_dir_map = {}
for item in edge_dir_emb:
    # print(item)
    edgeid_node_dir_map[item[0]] = item
    # break
    # print(len(item[4].append(item[5])))
    # break

In [37]:
len(edgeid_node_dir_map)
# edgeid_node_dir_map
# """
# 0: array([   0,    0, 4046,  106,    2]),
#  1: array([    1,     0, 29680,   359,     7]),
#  2: array([   2,    0, 2488,  191,    4]),
#  3: array([    3,     1, 11091,   317,     7]),
#  4: array([    4,     1, 33037,   268,     5]),
#  5: array([    5,     2, 11090,     1,     0]),
#  6: array([    6,     2, 16079,   267,     5]),
#  7: array([  7,   3,   4, 358,   7]),
#  8: array([   8,    3, 2349,  176,    3]),
#  9: array([   9,    3, 6167,  268,    5]),
# #  10: array([ 10,   4,   3, 178,   3]),
# """
np.save("./1114//edgeid_node_dir_map_eid.npy",edgeid_node_dir_map)

# 处理数据集

In [38]:
import os
path = "../../fmm/bj-output-omp-clean/"
files = os.listdir(path)
# files.sort(key=lambda x:int(x.split('.')[0]))

record_total = 0
len_total = 0

max_len = 0
min_len = 10000


# 保存子轨迹 大于等于15
traj = []


# exit()
for file in files:
    # print(file)
    sub_traj_mapped = pd.read_csv(path+"{}".format(file),delimiter=";")
    record_total = record_total + sub_traj_mapped.shape[0]
    # print(sub_traj_mapped)
    # break
    for index,row in sub_traj_mapped.iterrows():
        # print(row[0].split("\"")[1])  # 
        # break
        if len(row[0].split("\""))==5:
            if len(row[0].split("\"")[1].split(","))>=15:
                traj.append(row[0].split("\"")[1]+","+file.split(".")[0])  # 加上车辆ID
                # traj.append()

In [ ]:
traj  # ['82397,352,32434,297,35115,33296,33299,1736,609,19166,86233,611,85744,26248,26247,33309,26245,26243,10'

In [39]:
id_ndoe_dir_traj = []

for item in traj:
    # print(item)
    
    sub_traj = [int(i) for i in item.split(",")]
    # print(sub_traj[:-1])

    # break
    # for index,location in enumerate(sub_traj):
        # sub_traj[index] = label_id[location]
    temp = []
    for edgeid in sub_traj[:-1]:
        # print(edgeid)
        temp.append(np.append(edgeid_node_dir_map[edgeid],sub_traj[-1]))
    id_ndoe_dir_traj.append(temp)
    # break

In [40]:
id_ndoe_dir_traj = np.array(id_ndoe_dir_traj)
# np.save("./id_ndoe_dir_traj.npy",id_ndoe_dir_traj)
id_ndoe_dir_traj.shape

e:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


(19111,)

## 分割 滑动窗口为5

In [41]:
traj_dataset = []
z = 5
for item in id_ndoe_dir_traj:
    # print(item)
    # print(len(item))
    """
    edgeid,node_v,node_id,angle,象限
    [[39330, 2382886309, 17409, 270, 6],....]
            21
    """
    # break
    for i  in range(len(item)):
        # print(item[i*z:15+i*z])
        if(len(item[i*z:15+i*z])==15):
            traj_dataset.append(item[i*z:15+i*z])
    # break

In [42]:
traj_dataset = np.array(traj_dataset)
traj_dataset.shape  # (102035, 15, 6)  edgeid,node_v,node_id,angle,象限,taxiid

(102035, 15, 6)

In [43]:
np.save("./1114//traj_dataset_window5_5fea_taxiid.npy",traj_dataset)

## 路网约束 node_num * 8 

In [26]:
edge_dir_emb = pd.read_csv("./1114/edge_dir_emb-t.csv").to_numpy()
print(edge_dir_emb)
"""
[[0, 25585055, 0, 533650673, 4046, 106, 2],
 [1, 25585055, 0, 8835880543, 29680, 359, 7],
 [2, 25585055, 0, 342791346, 2488, 191, 4],
 [3, 25585083, 1, 1414035094, 11091, 317, 7],
 [4, 25585083, 1, 9378036221, 33037, 268, 5],
 [5, 25585092, 2, 1414035092, 11090, 1, 0],
 [6, 25585092, 2, 2099138477, 16079, 267, 5],
"""
node_mask = {}
for item in edge_dir_emb:
    # print(item)
    if item[1] not in node_mask:
        letter = [0 for _ in range(8)]
        letter[item[4]] = 1
        node_mask[item[1]] = letter
    else:
        node_mask[item[1]][item[4]] = 1
    
    # break

[[    0     0  4046   106     2]
 [    1     0 29680   359     7]
 [    2     0  2488   191     4]
 ...
 [86313 37484 36319    36     0]
 [86314 37484 37485   127     2]
 [86315 37485 37484   307     6]]


In [27]:
node_mask
np.save("./1114/node_mask.npy",node_mask)

In [26]:
dataset = np.load("./1114/traj_dataset_window5_5fea.npy",allow_pickle=True)